In [1]:
%matplotlib notebook
from pathlib import Path
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from celluloid import Camera
from IPython.display import HTML

In [2]:
root = Path('Dataset/imagesTr/')
label = Path('Dataset/labelsTr')

We basically are loading a sample image, and it's corresponding label, plotting it as an example
just to get familiar with the data.

In [3]:
# If this is our path:
# Dataset\imagesTr\la_003.nii.gz
# We change it to
# Dataset\labelsTr\la_003.nii.gz
# to access the labels of that image
def change_img_to_label_path(path):
    parts = list(path.parts)
    parts[parts.index('imagesTr')] = 'labelsTr'
    return Path(*parts)

In [4]:
# Getting the first path as a sample
sample_path = list(root.glob('la*'))[0]
sample_path_label = change_img_to_label_path(sample_path)

In [5]:
sample_path, sample_path_label

(WindowsPath('Dataset/imagesTr/la_003.nii.gz'),
 WindowsPath('Dataset/labelsTr/la_003.nii.gz'))

In [6]:
data = nib.load(sample_path)
label = nib.load(sample_path_label)

# mri is our data and mask is the highlighted parts we see sometimes
mri = data.get_fdata()
mask = label.get_fdata().astype(np.uint8)

In [7]:
nib.aff2axcodes(data.affine)

('R', 'A', 'S')

In [8]:
mri.shape

(320, 320, 130)

In [9]:
fig = plt.figure()
camera = Camera(fig)
for i in range(mri.shape[2]):
    
    # Looping through all 110 slices
    plt.imshow(mri[:, :, i], cmap='gray')
    # Making areas that are not of interest transparent with mask[:, :, i] == 0
    # And color the interest parts with mask[:, :, i]
    mask_ = np.ma.masked_where(mask[:, :, i] == 0, mask[:, :, i])
    
    plt.imshow(mask_, alpha=0.5)
    # Take a snapshot to create an animation
    camera.snap()
    
# Creating animation
animation = camera.animate()

<IPython.core.display.Javascript object>

In [10]:
HTML(animation.to_html5_video())

In [11]:
def normalize(full_volume):
    mu = full_volume.mean()
    std = np.std(full_volume)
    normalized = (full_volume - mu) / std
    return normalized


def standardize(normalized):
    standardized = (normalized - normalized.min()) / (normalized.max() - normalized.min())
    return standardized

In [12]:
all_files = list(root.glob('la*'))
len(all_files)

20

In [13]:
save_root = Path('Preprocessed')

for counter, path_to_mri_data in enumerate(tqdm(all_files)):

    path_to_label = change_img_to_label_path(path_to_mri_data)

    mri = nib.load(path_to_mri_data)
    assert nib.aff2axcodes(mri.affine) == ('R', 'A', 'S')
    mri_data = mri.get_fdata()
    label_data = nib.load(path_to_label).get_fdata().astype(np.uint8)

    # Cropping from top-down and left-right
    mri_data = mri_data[32:-32, 32:-32]
    label_data = label_data[32:-32, 32:-32]

    normalized_mri_data = normalize(mri_data)
    standardized_mri_data = standardize(normalized_mri_data)

    # Train-val split
    if counter < 17:
        current_path = save_root / 'train' / str(counter)
    else:
        current_path = save_root / 'val' / str(counter)

    # Looping from the last axis
    for i in range(standardized_mri_data.shape[-1]):
        # We w
        slice = standardized_mri_data[:, :, i]
        mask = label_data[:, :, i]
        
        slice_path = current_path / 'data'
        mask_path = current_path / 'masks'
        
        slice_path.mkdir(parents=True, exist_ok=True)
        mask_path.mkdir(parents=True, exist_ok=True)
        
        np.save(slice_path/str(i), slice)
        np.save(mask_path/str(i), slice)

  0%|          | 0/20 [00:00<?, ?it/s]

In [14]:
path = Path('Preprocessed/train/0')

In [15]:
# Opening a file for example
file = '50.npy'
slice = np.load(path/'data'/file)
mask = np.load(path/'masks'/file)

In [16]:
plt.figure()
plt.imshow(slice, cmap='gray')
mask_ = np.ma.masked_where(mask==0, mask)
plt.imshow(mask_, alpha=0.5)

<IPython.core.display.Javascript object>